# **Fera Formidável 4.15**

* Caio M. C. Ruas - RM: 24010

## Introdução

O objetivo dessa fera é implementar o operador genético de migração no código de algoritmo genético desenvolvido nesta disciplina (isto é, não é para usar o DEAP) e aplicá-lo para resolver algum problema de otimização, mostrando-o em ação.

Para testa-lo, vamos usar um problema em que seja necessário minimizar uma simples função matemática que contém múltiplos mínimos locais:

$$f(x,y) = \frac{x + y}{2} + \cos(10x) + \cos(10y)$$

Essa função é boa para o teste pois é extremamente simples, sendo fácil de entender, mas os múltiplos mínimos podem dificultar a convergência de algoritmos genéticos tradicionais. A ideia é que o operador de migração possa ajudar a escapar desses mínimos locais e encontrar uma solução melhor. Logo após a introdução, há um código demonstrando o gráfico da função para melhor compreensão.

A ideia de um operador de migração é simular a troca de informações entre populações diferentes, permitindo que soluções boas de uma população possam influenciar outras populações, aumentando a diversidade genética e potencialmente melhorando a qualidade das soluções encontradas. Isso não faz tanto sentido com os algoritmos aplicados até agora, pois em todos tinhamos apenas uma população sendo evoluída $^{[Obs]}$. Portanto, para aplicar esse operador, também precisamos modificar o algoritmo para trabalhar com múltiplas populações.

Para isso, vamos criar uma classe `Nacao` que irá gerenciar uma população, com seu próprio conjunto de indivíduos. A classe terá métodos para inicializar as populações, aplicar seleção, cruzamento e mutação, e realizar a migração entre a população de outras nações.

Obs: O conceito de migração também pode ser aplicado a uma única população, caso em que indivíduos novos são introduzidos a cada geração $^{[1]}$. Mas aqui vamos focar na migração entre populações diferentes.

In [19]:
import numpy as np
import plotly.graph_objects as go

def f(x, y):
    return (x + y) / 2 + np.cos(10 * x) + np.cos(10 * y)

x_vals = np.linspace(0, 5, 100)
y_vals = np.linspace(0, 5, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f(X, Y)

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')])

global_min_x = 0.314
global_min_y = 0.314
global_min_z = f(global_min_x, global_min_y)

fig.add_trace(go.Scatter3d(
    x=[global_min_x], 
    y=[global_min_y], 
    z=[global_min_z + 0.1],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='x'
    ),
    name='Mínimo Global (Aprox.)'
))


fig.update_layout(
    title='Problema do "Vale Inclinado com Relevo" 2D',
    scene=dict(
        xaxis_title='Eixo X',
        yaxis_title='Eixo Y',
        zaxis_title='f(x, y)'
    ),
    autosize=False,
    width=800,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()

Podemos perceber pelo gráfico a ideia disposta na introdução dos minimos locais e globais. Novamente, ideia é que o operador de migração possa ajudar a escapar desses mínimos locais e encontrar uma solução melhor.

## Bibliotecas e definições iniciais

Nessa célula, importamos as bibliotecas necessárias.

In [20]:
import random
import time

random.seed(8)

## Funções auxiliares

Aqui, definimos as funções auxiliares para o problema: as funções que geram indivíduos e populações iniciais e a função de avaliação. Essas funções são essenciais para o funcionamento do algoritmo genético.

In [21]:
def criar_gene():
    """Gera um único gene aleatório entre 0 e 50."""

    return random.uniform(0, 50)

def individuo_vales_inclinados_2d():
    """
    Gera um indivíduo aleatório para o problema 2D.
    O indivíduo é representado por dois genes: [x, y].
    """

    return [criar_gene(), criar_gene()]

def populacao_vales_inclinados_2d(tamanho_populacao):
    """Gera uma população inicial aleatória para o problema 2D."""
    return [individuo_vales_inclinados_2d() for _ in range(tamanho_populacao)]

def avaliacao_vales_inclinados_2d(individuo):
    """
    Avalia um indivíduo no problema do "Vale Inclinado com Relevo" 2D.
    """
    x = individuo[0]
    y = individuo[1]
    
    return (x + y) / 2 + np.cos(10 * x) + np.cos(10 * y)

## Funções do algoritmo genético

Aqui, definimos as funções necessárias para a estrutura do algoritmo genético, sendo essas as funções de seleção, cruzamento e mutação.

In [ ]:
# --- FUNÇÃO DE SELEÇÃO ---


def selecao_torneio(populacao, tamanho_torneio=2):
    """
    Seleciona indivíduos da população usando o método de torneio.
    """
    torneio = random.sample(populacao, tamanho_torneio)
    vencedor = min(torneio, key=avaliacao_vales_inclinados_2d)
    return vencedor

# --- FUNÇÕES DE CRUZAMENTO E MUTAÇÃO ---

def cruzamento_vales_inclinados(pai, mae):
    """
    Realiza o cruzamento entre dois indivíduos (pai e mãe) para gerar um novo indivíduo.
    """
    filho = []
    for gene_pai, gene_mae in zip(pai, mae):
        # Média dos genes do pai e da mãe
        novo_gene = (gene_pai + gene_mae) / 2
        filho.append(novo_gene)
    return filho

def mutacao_vales_inclinados_2d(individuo, taxa_mutacao=0.1):
    """
    Aplica uma mutação a um indivíduo para o problema 2D.
    """
    novo_individuo = []
    for gene in individuo:
        if random.random() < taxa_mutacao:
            # MUDANÇA: Usa a nova função para criar um gene.
            novo_gene = criar_gene() 
            novo_individuo.append(novo_gene)
        else:
            novo_individuo.append(gene)
    return novo_individuo

Agora, tem um ponto chave do funcionamento do nosso código. Para conseguir implementar o operador de migração, precisamos modificar a estrutura do algoritmo genético para trabalhar com múltiplas populações. Para isso, aqui criaremos a classe `Nacao` que irá gerenciar uma população, com seu próprio conjunto de indivíduos. A classe terá métodos para inicializar as populações, aplicar seleção, cruzamento e mutação, e realizar a migração entre a população de outras nações, como dito anteriormente.

In [23]:
class Nacao:
    """
    Classe que representa uma nação de indivíduos no contexto do algoritmo genético.
    
    A nação é composta por uma população de indivíduos que evolui independentemente, 
    mas tem métodos para enviar e receber migrantes de outras nações.
    """
    
    def __init__(self, nome_nacao, tamanho_populacao=50, taxa_mutacao=0.1, taxa_cruzamento=0.7):
        self.nome = nome_nacao
        self.tamanho_populacao = tamanho_populacao
        self.taxa_mutacao = taxa_mutacao
        self.taxa_cruzamento = taxa_cruzamento
        self.populacao = populacao_vales_inclinados_2d(tamanho_populacao)
        self.melhor_candidato = None
        self.melhor_candidato_global = None
        self.melhor_fitness_global = float('inf')
        self.n_melhores = []
    
    def avaliacao(self):
        """
        Avalia todos os indivíduos da população e retorna uma lista com os valores de fitness.
        """
        return [avaliacao_vales_inclinados_2d(individuo) for individuo in self.populacao]
    
    def melhor_individuo(self):
        """
        Encontra o melhor indivíduo da população com base na avaliação (o de menor fitness).
        """
        fitness = self.avaliacao()

        melhor_index = np.argmin(fitness)
        self.melhor_candidato = self.populacao[melhor_index]
        return self.melhor_candidato
    
    def melhores_individuos(self, n=10):
        """
        Encontra os n melhores indivíduos da população (os de menor fitness).
        """
        fitness = self.avaliacao()

        indices_melhores = np.argsort(fitness)[:n]
        self.n_melhores = [self.populacao[i] for i in indices_melhores]
        return self.n_melhores
    
    def melhor_individuo_global(self):
        """
        Encontra e armazena o melhor indivíduo encontrado até agora (o de menor fitness).
        """
        fitness = self.avaliacao()
        melhor_index = np.argmin(fitness)
        melhor_fitness = fitness[melhor_index]
        
        if melhor_fitness < self.melhor_fitness_global:
            self.melhor_fitness_global = melhor_fitness
            self.melhor_candidato_global = self.populacao[melhor_index]
        
        return self.melhor_candidato_global
    
    def enviar_migrantes(self, outra_nacao, taxa_migracao=0.1):
        """
        Envia os melhores indivíduos como migrantes para outra nação.
        """
        num_migrantes = int(self.tamanho_populacao * taxa_migracao)

        migrantes = self.melhores_individuos(n=num_migrantes)
        outra_nacao.receber_migrantes(migrantes)

    def receber_migrantes(self, migrantes):
        """
        Recebe migrantes e os adiciona à população, depois remove os piores para manter o tamanho.
        """
        self.populacao.extend(migrantes)
        
        populacao_ordenada = sorted(self.populacao, key=avaliacao_vales_inclinados_2d)
        self.populacao = populacao_ordenada[:self.tamanho_populacao]
        
    def evoluir(self, geracoes=100):
        """
        Evolui a população por um número de gerações especificado.
        """
        for _ in range(geracoes):
            nova_populacao = []
            
            melhores = self.melhores_individuos(n=2)
            nova_populacao.extend(melhores)
            
            while len(nova_populacao) < self.tamanho_populacao:

                pai = selecao_torneio(self.populacao)
                mae = selecao_torneio(self.populacao)
                
                if random.random() < self.taxa_cruzamento:
                    filho = cruzamento_vales_inclinados(pai, mae)
                else:
                    filho = pai
                
                filho = mutacao_vales_inclinados_2d(filho, self.taxa_mutacao)
                nova_populacao.append(filho)
            
            self.populacao = nova_populacao
            self.melhor_individuo()
            self.melhor_individuo_global()

## Aplicando o algoritmo genético

Nessa etapa, juntamos as funções definidas anteriormente para estruturar e aplicar a função do algoritmo genético em sí.

Vamos criar mais 3 funções apenar para as ações de interagir com as nações, como criá-las, evolui-las e migrar os indivíduos entre elas. Essas funções serão responsáveis por gerenciar as populações e aplicar o operador de migração.

In [ ]:
def criar_nacoes(num_nacoes, tamanho_populacao):
    """
    Cria uma lista de nações com o tamanho de população especificado.
    
    """
    return [Nacao(f"Nação {i+1}", tamanho_populacao) for i in range(num_nacoes)]

def evoluir_nacoes(nacoes, num_geracoes):
    """
    Evolui uma lista de nações por um número especificado de gerações.

    """
    for nacao in nacoes:
        nacao.evoluir(num_geracoes)
        print(f"{nacao.nome} - Melhor Fitness Global: {nacao.melhor_fitness_global}, Melhor Indivíduo Global: {nacao.melhor_candidato_global}")

def migrar_nacoes(nacoes, taxa_migracao=0.1):
    """
    Realiza a migração entre nações com base na taxa de migração especificada.
    
    """
    for i in range(len(nacoes)):
        for j in range(i + 1, len(nacoes)):
            nacoes[i].enviar_migrantes(nacoes[j], taxa_migracao)
            nacoes[j].enviar_migrantes(nacoes[i], taxa_migracao)

Quase pronto! Agora vem um bônus: seria legal comparar a evolução de uma população com e sem o operador de migração. Vamos fazer isso! Então será bom ter uma função que execute o algoritmo genético, para facilitar a aplicação dpeois.

In [ ]:
# --- FUNÇÃO PRINCIPAL DO EXPERIMENTO ---

def executar_experimento(num_nacoes, tamanho_pop, max_geracoes, fitness_alvo, com_migracao=False):
    """
    Executa uma simulação completa do algoritmo genético, com ou sem migração.
    """
    # Cria as nações para esta execução
    nacoes = criar_nacoes(num_nacoes, tamanho_pop)
    
    # Loop principal de evolução
    for geracao_atual in range(max_geracoes):
        
        # 1. Evolui todas as nações por uma geração
        for nacao in nacoes:
            nacao.evoluir(geracoes=1) # A evolução é passo a passo

        # 2. Realiza a migração, se habilitada
        # A migração ocorrerá a cada 20 gerações
        if com_migracao and (geracao_atual + 1) % 20 == 0:
            migrar_nacoes(nacoes, taxa_migracao=0.1)

        # 3. Verifica o critério de parada
        melhor_fitness_da_rodada = float('inf')
        for nacao in nacoes:
            if nacao.melhor_fitness_global < melhor_fitness_da_rodada:
                melhor_fitness_da_rodada = nacao.melhor_fitness_global
        
        # Se qualquer nação atingiu o alvo, paramos e retornamos a geração atual
        if melhor_fitness_da_rodada <= fitness_alvo:
            print(f"--> Alvo de fitness atingido na geração {geracao_atual + 1}! Fitness: {melhor_fitness_da_rodada:.4f}")
            return geracao_atual + 1

    # Se o loop terminar, o número máximo de gerações foi atingido
    print(f"--> Máximo de {max_geracoes} gerações atingido.")
    return max_geracoes

Agora sim!! Vamos testar um algoritmo evoluindo uma única população, outro evoluindo múltiplas populações sem migração e outro evoluindo múltiplas populações **com migração**. Vamos executar cada um desses 15 vezes e tirar uma média de quanto tempo cada um demorou para encontrar uma solução satisfatória, ou seja, o fitness alvo, que é o mínimo global da função que mostramos.

In [26]:
# --- CONFIGURAÇÃO E EXECUÇÃO DAS SIMULAÇÕES ---

# Parâmetros do experimento
NUM_EXECUCOES = 15
TAMANHO_POPULACAO = 50
MAX_GERACOES = 1000
FITNESS_ALVO = -1.65

resultados_populacao_unica = []
resultados_sem_migracao = []
resultados_com_migracao = []

print("--- Iniciando Simulações Comparativas ---")

for i in range(NUM_EXECUCOES):
    print(f"\n--- Execução {i + 1}/{NUM_EXECUCOES} ---")
    
    # Cenário 1: Uma única população (SEM migração)
    print("Rodando cenário SEM migração - População única...")
    random.seed(i)
    tempo_inicio = time.time()
    geracoes = executar_experimento(1, 60, MAX_GERACOES, FITNESS_ALVO, com_migracao=False)
    resultados_populacao_unica.append(geracoes)
    tempo_fim = time.time()
    print(f"Tempo decorrido: {tempo_fim - tempo_inicio:.2f} segundos.")

    # Cenário 2: Nações isoladas (SEM migração)
    print("Rodando cenário SEM migração - População múltipla...")
    random.seed(i)
    tempo_inicio = time.time()
    geracoes = executar_experimento(5, TAMANHO_POPULACAO, MAX_GERACOES, FITNESS_ALVO, com_migracao=False)
    resultados_sem_migracao.append(geracoes)
    tempo_fim = time.time()
    print(f"Tempo decorrido: {tempo_fim - tempo_inicio:.2f} segundos.")

    # Cenário 3: Nações conectadas (COM migração)
    print("\nRodando cenário COM migração...")
    random.seed(i)
    tempo_inicio = time.time()
    geracoes = executar_experimento(5, TAMANHO_POPULACAO, MAX_GERACOES, FITNESS_ALVO, com_migracao=True)
    resultados_com_migracao.append(geracoes)
    tempo_fim = time.time()
    print(f"Tempo decorrido: {tempo_fim - tempo_inicio:.2f} segundos.")

--- Iniciando Simulações Comparativas ---

--- Execução 1/15 ---
Rodando cenário SEM migração - População única...
--> Alvo de fitness atingido na geração 160! Fitness: -1.6670
Tempo decorrido: 0.18 segundos.
Rodando cenário SEM migração - População múltipla...
--> Alvo de fitness atingido na geração 159! Fitness: -1.6675
Tempo decorrido: 0.58 segundos.

Rodando cenário COM migração...
--> Alvo de fitness atingido na geração 81! Fitness: -1.6873
Tempo decorrido: 0.33 segundos.

--- Execução 2/15 ---
Rodando cenário SEM migração - População única...
--> Alvo de fitness atingido na geração 153! Fitness: -1.6819
Tempo decorrido: 0.13 segundos.
Rodando cenário SEM migração - População múltipla...
--> Alvo de fitness atingido na geração 117! Fitness: -1.6880
Tempo decorrido: 0.45 segundos.

Rodando cenário COM migração...
--> Alvo de fitness atingido na geração 117! Fitness: -1.6880
Tempo decorrido: 0.45 segundos.

--- Execução 3/15 ---
Rodando cenário SEM migração - População única...
--> 

Legal, parece que foi! Vamos printar os resultados e ver como cada abordagem se saiu.

In [27]:
# --- APRESENTAÇÃO DOS RESULTADOS FINAIS ---

print(f"\n--- Resultados Finais (Médias de {NUM_EXECUCOES} Execuções) ---")

media_populacao_unica = np.mean(resultados_populacao_unica)
media_sem_migracao = np.mean(resultados_sem_migracao)
media_com_migracao = np.mean(resultados_com_migracao)

print(f"\nMédia de gerações para atingir o alvo SEM migração (População única): {media_populacao_unica:.2f}")
print(f"Média de gerações para atingir o alvo SEM migração (População múltipla): {media_sem_migracao:.2f}")
print(f"Média de gerações para atingir o alvo COM migração: {media_com_migracao:.2f}\n")

if media_com_migracao < media_sem_migracao:
    print("Conclusão: O algoritmo com migração convergiu significativamente mais rápido.")
else:
    print("Conclusão: Neste teste, a migração não demonstrou uma vantagem clara na velocidade de convergência.")


--- Resultados Finais (Médias de 15 Execuções) ---

Média de gerações para atingir o alvo SEM migração (População única): 199.73
Média de gerações para atingir o alvo SEM migração (População múltipla): 96.87
Média de gerações para atingir o alvo COM migração: 71.40

Conclusão: O algoritmo com migração convergiu significativamente mais rápido.


Bom, parece que realmente o operador de migração ajudou a melhorar a convergência do algoritmo genético, permitindo que ele encontrasse soluções melhores em menos tempo. Isso demonstra a eficácia do operador de migração em problemas com múltiplos mínimos locais.

## Conclusão

Foi possível implementar o operador de migração no código de algoritmo genético desenvolvido nesta disciplina e aplicá-lo para resolver um problema de otimização. O operador de migração demonstrou ser eficaz em melhorar a convergência do algoritmo, permitindo que ele encontrasse soluções melhores em menos tempo.

## Referências

$^{[1]}$ GLADWIN, D.; STEWART, P.; STEWART, J. A controlled migration genetic algorithm operator for hardware-in-the-loop experimentation. Engineering Applications of Artificial Intelligence, v. 24, n. 4, p. 586–594, jun. 2011. 

$^{[2]}$ KORKMAZ TAN, R.; BORA, Ş. Adaptive Genetic Algorithm Renewed by Migration Operator. European Journal of Science and Technology, 30 jun. 2021. 